In [5]:
/*
 * Si Python et/ou R ne fonctionnent pas, voyez: https://github.com/dotnet/interactive/blob/HEAD/docs/jupyter-in-polyglot-notebooks.md
 *
 * Enlevez les commentaires qui suivent afin de connecter Python et R.
 */

// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3

// #!connect jupyter --kernel-name Rkernel --kernel-spec ir

In [28]:
#r "nuget: Microsoft.Extensions.Http, 8.0.0"

#r "nuget: GBX.NET, 2.0.9"
#r "nuget: GBX.NET.LZO, 2.1.1"
#r "nuget: GBX.NET.ZLib, 1.1.0"

Installed Packages GBX.NET, 2.0.9 GBX.NET.LZO, 2.1.1 GBX.NET.ZLib, 1.1.0 Microsoft.Extensions.Http, 8.0.0

## Assigner les valeurs d'échantillonage

Modifiez les valeurs d'échantillonage afin de modifier la taille, le nombre de records sur chaque carte, et cetera.

In [293]:
public class SampleSettings
{
  public static int minRecords = 5;

  public static int sampleSizePerTag = 1;
}

## Préparation pour l'utilisation du API de TMNF-X (Trackmania Nations Forever Exchange), ansi que de leur site normal

In [1]:

public enum Order1 {
  None = 0,
  UploadDateOldest = 1,
  UploadDateNewestDefault = 2,
  UpdateDateOldest = 3,
  UpdateDateNewest = 4,
  AwardCountAscending = 5,
  AwardCountDescending = 6,
  CommentCountAscending = 7,
  CommentCountDescending = 8,
  ActivityOldest = 9,
  ActivityNewest = 10,
  TrackNameAZ = 11,
  TrackNameZA = 12,
  AuthorNameAZ = 13,
  AuthorNameZA = 14,
  DifficultyAscending = 15,
  DifficultyDescending = 16,
  DownloadCountAscending = 17,
  DownloadCountDescending = 18,
  TrackValueAscending = 19,
  TrackValueDescending = 20,
  AwardsthisweekAscending = 21,
  AwardsthisweekDescending = 22,
  AwardsthismonthAscending = 23,
  AwardsthismonthDescending = 24,
  LastAwardDateOldest = 25,
  LastAwardDateNewest = 26,
  WorldRecordsetOldest = 27,
  WorldRecordsetNewest = 28,
  PersonalrecordsetOldestLoginrequired = 29,
  PersonalrecordsetNewestLoginrequired = 30,
  PersonalawardsetOldestLoginrequired = 31,
  PersonalawardsetNewestLoginrequired = 32,
  PersonaldownloadOldestLoginrequired = 33,
  PersonaldownloadNewestLoginrequired = 34,
  PersonalcommentsetOldestLoginrequired = 35,
  PersonalcommentsetNewestLoginrequired = 36,
  UserRecordPositionLowest = 37,
  UserRecordPositionHighest = 38,
}

public enum Tag {
  Normal = 0,
  //Stunt = 1,
  //Maze = 2,
  Offroad = 3,
  //Laps = 4,
  Fullspeed = 5,
  //LOL = 6,
  Tech = 7,
  SpeedTech = 8,
  //RPG = 9,
  PressForward = 10,
  //Trial = 11,
  Grass = 12,
}

public enum PrimaryType {
  Race = 0,
  Puzzle = 1,
  Platform = 2,
  Stunts = 3,
  Shortcut = 4,
  Laps = 5,
}

public enum LBType {
  Allreplaytypes = -1,
  Normal = 0,
  Classic = 1,
  Nadeo = 2,
  Useless = 3,
}

public enum Difficulty {
  Beginner = 0,
  Intermediate = 1,
  Expert = 2,
  Lunatic = 3,
}

public enum Routes {
  Single = 0,
  Multiple = 1,
  Symmetrical = 2,
}

In [2]:
#nullable enable

public class TMXResponses
{
  public static DateTime newestMapAt { get; set; }
  public static DateTime oldestMapAt { get; set; }

  public class ParsedResponse<T>
  {
    public Boolean More { get; set; }
    public T[]? Results { get; set; }
  }

  public class MapResponse
  {
      public int? TrackId { get; set; }
      public string? TrackName { get; set; }
      public string? UId { get; set; }
      public int? AuthorTime { get; set; }
      public Uploader? Uploader { get; set; }
      public DateTime? UploadedAt { get; set; }
      public DateTime? UpdatedAt { get; set; }
      public int? PrimaryType { get; set; } // enum
      public int? TrackValue { get; set; }
      public int? Style { get; set; } // enum
      public int? Routes { get; set; } // enum
      public int? Difficulty { get; set; }
      public int? Environment { get; set; } // enum
      public int? Car { get; set; } // enum
      public WRReplay? WRReplay { get; set; }
  }

  public class Uploader
  {
    public int? UserId { get; set; }
    public string? Name { get; set; }
  }

  public class WRReplay
  {
    public int ReplayId { get; set; }
    public int ReplayTime { get; set; }
  }

  public class ReplayResponse
  {
      public Int64? ReplayId { get; set; }
      public User? User { get; set; }
      public Int32? ReplayTime { get; set; }
      public Int32? ReplayScore { get; set; }
      public Int32? ReplayRespawns { get; set; }
      public Int32? Score { get; set; }
      public Int32? Position { get; set; }
      public Int32? IsBest { get; set; }
      public Int32? IsLeaderboard { get; set; }
      public DateTime? TrackAt { get; set; }
      public DateTime? ReplayAt { get; set; }
  }

  public class User
  {
    public int? UserId { get; set; }
    public String? Name { get; set; }
  }
}

In [26]:
#nullable enable

public class TMXInputs
{
  private static String inputParametersToString<T>(T inputParametersObject)
  {
    // Voir https://stackoverflow.com/questions/4276566/how-can-you-loop-over-the-properties-of-a-class

    StringBuilder paramaterString = new StringBuilder();

    Boolean needsAnd = false;
    foreach(var prop in typeof(T).GetProperties()) {
      object? value = prop.GetValue(inputParametersObject, null);

      if (value == null) { continue; }

      if (needsAnd) {
        paramaterString.Append("&");
      } else { needsAnd = true; }

      if (prop.Name == "fields") {
        paramaterString.Append($"fields={String.Join("%2C", (String[]) value)}");
        continue;
      }

      // If DateTime
      if (value is DateTime) {
        string dateString = ((DateTime) value).ToString("s", System.Globalization.CultureInfo.InvariantCulture);

        paramaterString.Append($"{prop.Name}={dateString}");
        continue;
      }

      // If Enum[]
      if (value.GetType().IsArray && (((Array) value).GetValue(0)??"NotEnum").GetType().IsEnum) {
        paramaterString.Append($"{prop.Name}=");

        int[] enumValue =  (int[]) value;

        paramaterString.Append(enumValue[0]);

        for (int i = 1; i < enumValue.Length; i++) {
          paramaterString.Append($"%2C{enumValue[i]}");
        }
        
        continue;
      }

      // If Enum
      if (value is Enum) {
        paramaterString.Append($"{prop.Name}={((Enum) value).ToString("D")}");
        continue;
      }

      paramaterString.Append($"{prop.Name}={value}");
    }
    
    return paramaterString.ToString();
  }

  public class MapInputsParameters {
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Search Order
      /// </summary>
    public Order1? order1 { get; set; }
      /// <summary>
      /// Amount of results (default: 40)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified TrackId and given order - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified TrackId and given order - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified TrackId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// Filter by TrackId's
      /// </summary>
    public Int64[,]? id { get; set; }
      /// <summary>
      /// Filter by UId or Track file signatures
      /// </summary>
    public String[,]? uid { get; set; }
      /// <summary>
      /// Filter by Tags[]
      /// </summary>
    public Tag[]? tag { get; set; }
      /// <summary>
      /// Filter by excluding Tags[]
      /// </summary>
    public Tag[]? etag { get; set; }
      /// <summary>
      /// true: track must have all specified tags, false: track must have one of the specified tags
      /// </summary>
    public Boolean taginclusive { get; } = true;
      /// <summary>
      /// Filter by PrimaryType
      /// </summary>
    public PrimaryType primarytype { get; } = PrimaryType.Race;
      /// <summary>
      /// Filter by min. UploadedAt
      /// </summary>
    public DateTime? uploadedafter { get; set; }
      /// <summary>
      /// Filter by max. UploadedAt
      /// </summary>
    public DateTime? uploadedbefore { get; set; }
      /// <summary>
      /// Filter by min. AuthorTime in ms
      /// </summary>
    public Int32? authortimemin { get; } = 30000;
      /// <summary>
      /// Filter by max. AuthorTime in ms
      /// </summary>
    public Int32? authortimemax { get; } = 120000;
      /// <summary>
      /// Filter by Routes
      /// </summary>
    public Routes routes { get; } = Routes.Single;
      /// <summary>
      /// 1: Track has at least one replay
      /// </summary>
    public Int32 inhasrecord { get; } = 1;
      /// <summary>
      /// 1: Track is environment mixed (Environment =/= Car)
      /// </summary>
    public Int32 inenvmix { get; } = 0;
      /// <summary>
      /// 1: Track requires to be played with TMUnlimiter, 0: vanilla game
      /// </summary>
    public Int32 inunlimiter { get; } = 0;

    public override string ToString(){
      return inputParametersToString<MapInputsParameters>(this);
    }  
  }

  public class ReplayInputsParameters {
      /// <summary>
      /// TM1X TrackId of track containing the replays
      /// </summary>
    public String? trackId { get; set; }
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Amount of results (default: 10)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified ReplayId - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified ReplayId - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified ReplayId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// 1: Returns only the best replay
      /// </summary>
    public Int32? best { get; set; } = 1;
      /// <summary>
      /// Filter by driver TM1X UserId
      /// </summary>
    public Int64? userid { get; set; }

    public override string ToString(){
      return inputParametersToString<ReplayInputsParameters>(this);
    }  
  }
}

In [ ]:
using GBX.NET;
using GBX.NET.Engines.Game;
using GBX.NET.Engines.Plug;

#nullable enable

public class DataContainers {
  public class Vector3 {
    public float X { get; set; } = 0;
    public float Y { get; set; } = 0;
    public float Z { get; set; } = 0;
  }

  public class SampleData {
      /// <summary>
      /// Temps actuel de l'échantillon, ayant comme unité 10 millisecondes
      /// </summary>
    public float? Time { get; set; }

      /// <summary>
      /// Révolutions par minute
      /// </summary>
    public float? RPM { get; set; }

      /// <summary>
      /// Vitesse vers l'avant de la voiture
      /// </summary>
    public float? SpeedForward { get; set; }
      /// <summary>
      /// Vitesse vers le coté de la voiture, soit perpendiculaire à la direction dont la voiture fait face
      /// </summary>
    public float? SpeedSideward { get; set; }

      /// <summary>
      /// Vecteur-vitesse angulaire en 3 dimensions
      /// </summary>
    public Vector3? AngularVelocity { get; set; } = new Vector3();

      /// <summary>
      /// Tangage de la voiture
      /// </summary>
    public Double? Pitch { get; set; }
      /// <summary>
      /// Lacet de la voiture
      /// </summary>
    public Double? Yaw { get; set; }
      /// <summary>
      /// Roulis de la voiture
      /// </summary>
    public Double? Roll { get; set; }

      /// <summary>
      /// Position de la voiture à un instant en 3 dimensions
      /// </summary>
    public Vector3? Position { get; set; } = new Vector3();

      /// <summary>
      /// Booléen de la présence d'un turbo
      /// </summary>
    public Boolean? IsTurbo { get; set; }
    
      /// <summary>
      /// Matériel de contact de la roue avant-droite
      /// </summary>
    public CPlugSurface.MaterialId? FRGroundContactMaterial { get; set; }
      /// <summary>
      /// Matériel de contact de la roue avant-gauche
      /// </summary>
    public CPlugSurface.MaterialId? FLGroundContactMaterial { get; set; }
      /// <summary>
      /// Matériel de contact de la roue arrière-droite
      /// </summary>
    public CPlugSurface.MaterialId? RRGroundContactMaterial { get; set; }
      /// <summary>
      /// Matériel de contact de la roue arrière-gauche
      /// </summary>
    public CPlugSurface.MaterialId? RLGroundContactMaterial { get; set; }

    ///////////////////////////////////////////////////////////// Non nécessaire
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue avant-droite
    //   /// </summary>
    // public float? FRDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue avant-gauche
    //   /// </summary>
    // public float? FLDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue arrière-droite
    //   /// </summary>
    // public float? RRDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue arrière-gauche
    //   /// </summary>
    // public float? RLDampenLen { get; set; }
  }

  public class ReplayData {
      /// <summary>
      /// L'étiquette du circuit.
      /// </summary>
    public String? Tag { get; set; }
      /// <summary>
      /// Durée totale du replay, ayant comme unité 10 millisecondes.
      /// </summary>
    public float? Duration { get; set; }
      /// <summary>
      /// Somme absolue de différences de positions par axe
      /// </summary>
    public Vector3? TotalDisplacement { get; set; } = new Vector3();
      /// <summary>
      /// Les échantillons du replay
      /// </summary>
    public List<SampleData>? Samples { get; set; }
  }
}

In [29]:
using System;
using System.Globalization;
using System.Net.Http;
using System.Net;
using System.Text.Json;
using GBX.NET;
using GBX.NET.Engines.Game;
using GBX.NET.Engines.Scene;
using GBX.NET.LZO;
using GBX.NET.ZLib;

#nullable enable

Gbx.LZO = new Lzo();
Gbx.ZLib = new ZLib();

public class TMXAPI
{
  public static HttpClient sharedClient = new()
  {
      BaseAddress = new Uri("https://tmnf.exchange"),
  };

  // private static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponse(TMXInputs.MapInputsParameters paramaters)
  // {
  //   using HttpResponseMessage response = await sharedClient.GetAsync($"api/tracks?{paramaters}"); // Note: If "parameters" can be chosen by the user, add sanitization

  //   Console.WriteLine($"Get: api/tracks?{paramaters}");
    
  //   response.EnsureSuccessStatusCode();
    
  //   var jsonResponse = await response.Content.ReadAsStringAsync();

  //   TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

  //   return mapResponse;
  // }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponseFromID(String[] fields, String id)
  {
    String relativeURL = $"api/tracks?fields={String.Join("%2C", fields)}&id={id}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine(relativeURL);
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

    return mapResponse;
  }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>?> getTopReplaysResponseFromMapID(String[] fields, String trackId)
  {
    TMXInputs.ReplayInputsParameters inputsParameters = new TMXInputs.ReplayInputsParameters{ trackId = trackId, fields = fields, count = SampleSettings.minRecords};

    String relativeURL = $"api/replays?{inputsParameters}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine(relativeURL);
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>? replayResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>>(jsonResponse);

    return replayResponse;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L200
  private static Uri? GetRequestUriFromResponse(HttpResponseMessage response)
  {
      if (response.RequestMessage is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request message. This is odd...");
          return null;
      }

      if (response.RequestMessage.RequestUri is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request URI. This is odd...");
          return null;
      }

      return response.RequestMessage.RequestUri;
  }

  // Voir https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L217
  private static string? GetTrackIdFromUri(Uri uri)
  {
      var trackId = uri.Segments.LastOrDefault();

      if (trackId is null) { return null; }

      return trackId;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs
  public static async Task<string?> getRandomMapID(TMXInputs.MapInputsParameters paramaters)
  {
    string? trackId = null;
    
    using var response = await sharedClient.SendAsync(new HttpRequestMessage(HttpMethod.Head, $"trackrandom?{paramaters}")); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine($"Head: trackrandom?{paramaters}");

    if (response.StatusCode == HttpStatusCode.NotFound)
    {
      // I have written something incorrectly

      throw new Exception("No map meets the filter; the programmer has written erronous parameters");
    }
    
    response.EnsureSuccessStatusCode();
    
    var requestUri = GetRequestUriFromResponse(response);

    if (requestUri is null)
    {
        Console.Error.WriteLine("RequestUri of /trackrandom is null");
        return null;
    }

    trackId = GetTrackIdFromUri(requestUri);

    return trackId;
  }

  public static async Task<CGameCtnReplayRecord> getReplayFilestream(string replayId) {
    String relativeURL = $"recordgbx/{replayId}";

    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL);

    response.EnsureSuccessStatusCode();
    
    var gbxResponse = Gbx.Parse<CGameCtnReplayRecord>(response.Content.ReadAsStream());

    return gbxResponse;
  }

  /// <summary>
  /// CETTE MÉTHODE NE PROCURE PAS L'ÉTIQUETTE
  /// </summary>
  public static async Task<DataContainers.ReplayData> getReplayData(string replayId) {
    DataContainers.ReplayData replayData = new DataContainers.ReplayData();

    CGameCtnReplayRecord replay = await getReplayFilestream(replayId); // Terrible error management

    CGameCtnGhost ghost = replay.GetGhosts().First<CGameCtnGhost>();

    replayData.Duration = ghost.RaceTime!.Value.TotalSeconds;

    List<DataContainers.SampleData> samples = new List<DataContainers.SampleData>();

    DataContainers.Vector3 displacement = new DataContainers.Vector3();
    foreach (CSceneVehicleCar.Sample sample in ghost.SampleData!.Samples) {

      DataContainers.SampleData sampleData = new DataContainers.SampleData();

      sampleData.Time = sample.Time.TotalSeconds;

      sampleData.RPM = sample.RPM;
      
      sampleData.SpeedForward = sample.SpeedForward;
      sampleData.SpeedSideward = sample.SpeedSideward;

      sampleData.AngularVelocity!.X = sample.AngularVelocity.X;
      sampleData.AngularVelocity.Y = sample.AngularVelocity.Y;
      sampleData.AngularVelocity.Z = sample.AngularVelocity.Z;

      // Voir https://stackoverflow.com/questions/54214698/quaternion-to-yaw-pitch-roll
      GBX.NET.Quat quatRotation = sample.Rotation;
      sampleData.Pitch = Math.Asin(-2.0*(quatRotation.X*quatRotation.Z - quatRotation.W*quatRotation.Y));
      sampleData.Yaw = Math.Atan2(2.0*(quatRotation.Y*quatRotation.Z + quatRotation.W*quatRotation.X), quatRotation.W*quatRotation.W - quatRotation.X*quatRotation.X - quatRotation.Y*quatRotation.Y + quatRotation.Z*quatRotation.Z);
      sampleData.Roll = Math.Atan2(2.0*(quatRotation.X*quatRotation.Y + quatRotation.W*quatRotation.Z), quatRotation.W*quatRotation.W + quatRotation.X*quatRotation.X - quatRotation.Y*quatRotation.Y - quatRotation.Z*quatRotation.Z);

      sampleData.Position!.X = sample.Position.X;
      sampleData.Position.Y = sample.Position.Y;
      sampleData.Position.Z = sample.Position.Z;

      sampleData.IsTurbo = sample.IsTurbo;
      
      sampleData.FRGroundContactMaterial = sample.FRGroundContactMaterial;
      sampleData.FLGroundContactMaterial = sample.FLGroundContactMaterial;
      sampleData.RRGroundContactMaterial = sample.RRGroundContactMaterial;
      sampleData.RLGroundContactMaterial = sample.RLGroundContactMaterial;

      /////////////////////////////////////////////// Non nécessaire
      // sampleData.FRDampenLen = sample.FRDampenLen;
      // sampleData.FLDampenLen = sample.FLDampenLen;
      // sampleData.RRDampenLen = sample.RRDampenLen;
      // sampleData.RLDampenLen = sample.RLDampenLen;

      displacement.X += Math.Abs(sample.Position.X);
      displacement.Y += Math.Abs(sample.Position.Y);
      displacement.Z += Math.Abs(sample.Position.Z);

      samples.Add(sampleData);
    }

    replayData.TotalDisplacement = displacement;

    replayData.Samples = samples;

    return replayData;
  }
}


## Sélection aléatoire stratifié de circuits et de _replays_

Pour sélectionner un circuit aléatoire, nous utilisons le sous-répertoire `/trackrandom` du site TMNF-X. Les `minRecords` _replays_ les plus rapides -- de joueurs différents -- sont ensuite cherchés. Si un nombre de _replays_ équivalent à `minRecords` sont trouvés, les identifiants du circuit et du record mondial sont enregistrés.

In [92]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;

public class RandomMap()
{
  // public static Dictionary<Tag, List<String[]>> chosenMaps = (Enum.GetValues<Tag>()).ToDictionary(key => key, value => new List<String[]>());

  public static List<DataContainers.ReplayData> allReplayData = new List<DataContainers.ReplayData>();

  public static async Task selectMapIds() {
    foreach (Tag tag in Enum.GetValues<Tag>()) {
    // Tag tag = Tag.Grass;

      Console.WriteLine("_______________________________________________________________________________________________________________");
      string chosenId = null;

      int remainingSamples = SampleSettings.sampleSizePerTag;
      for (int i = SampleSettings.sampleSizePerTag * 20; i > 0 && remainingSamples > 0; i--) {
        Console.WriteLine("------");
        chosenId = await TMXAPI.getRandomMapID(new TMXInputs.MapInputsParameters{tag = [tag]});

        TMXResponses.ParsedResponse<TMXResponses.MapResponse> mapResponse = await TMXAPI.getMapResponseFromID(["TrackValue"], chosenId);

        Console.WriteLine($"Queried map id: {chosenId}");

        TMXResponses.ParsedResponse<TMXResponses.ReplayResponse> replaysResponse = await TMXAPI.getTopReplaysResponseFromMapID(["ReplayId","User.Name"], chosenId);

        if(replaysResponse.Results.Length < SampleSettings.minRecords) { continue; }
        remainingSamples--;

        // replaysResponse.Display();

        Console.WriteLine($"Fastest replay id: {replaysResponse.Results[0].ReplayId}");

        // chosenMaps[tag].Add(new String[]{chosenId, replaysResponse.Results[0].ReplayId.ToString()});

        /////////////////////

        DataContainers.ReplayData replayData = await TMXAPI.getReplayData(replaysResponse.Results[0].ReplayId.ToString());
        replayData.Tag = tag.ToString();

        allReplayData.Add(replayData);
      }
    }
  }
}

await RandomMap.selectMapIds();

RandomMap.allReplayData.Display();

string json = JsonSerializer.Serialize(RandomMap.allReplayData);

json.Display();

File.WriteAllText(@"replay-data.json", json);

_______________________________________________________________________________________________________________
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=1294985
Queried map id: 1294985
api/replays?trackId=1294985&fields=ReplayId%2CUser.Name&count=5&best=1
Fastest replay id: 2025514
_______________________________________________________________________________________________________________
------
Head: trackrandom?tag=3&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=3479531
Queried map id: 3479531
api/replays?trackId=3479531&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=3&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=Track

index value 0 Submission#31+DataContainers+ReplayData Tag Normal Duration 40.58 TotalDisplacement Submission#31+DataContainers+Vector3 X 176020.38 Y 19001.957 Z 212913.38 Samples index value 0 Submission#31+DataContainers+SampleData Time 0 RPM 0 SpeedForward -0.43945312 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X 0 Y 0 Z 0 Pitch 1.5704510598101804 Yaw 0.5881558569344463 Roll 0.5881227113971044 Position Submission#31+DataContainers+Vector3 X 491.2 Y 50.21 Z 624 IsTurbo False FRGroundContactMaterial Concrete FLGroundContactMaterial Concrete RRGroundContactMaterial Concrete RLGroundContactMaterial Concrete 1 Submission#31+DataContainers+SampleData Time 0.1 RPM 799.72534 SpeedForward 4.9987793 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X -2.5083084E-09 Y -0 Z 0.05738341 Pitch 1.5704510598101804 Yaw 0.5791106421515969 Roll 0.5815112515537003 Position Submission#31+DataContainers+Vector3 X 491.2554 Y 50.096943 Z 624 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 2 Submission#31+DataContainers+SampleData Time 0.2 RPM 1799.9542 SpeedForward 10.437231 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X -0.021127421 Y -0.0015707659 Z 0.020925304 Pitch 1.569651205697217 Yaw -1.5006686982952717 Roll -1.4966586240371982 Position Submission#31+DataContainers+Vector3 X 491.4678 Y 50.007374 Z 624.0005 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 3 Submission#31+DataContainers+SampleData Time 0.3 RPM 1679.5605 SpeedForward 15.8754635 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X -0.0013499415 Y -6.684142E-05 Z 0.0037454765 Pitch 1.569104870726493 Yaw -1.5257376331766357 Roll -1.5214993057586519 Position Submission#31+DataContainers+Vector3 X 491.8381 Y 50.008217 Z 624.0008 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 4 Submission#31+DataContainers+SampleData Time 0.4 RPM 2042.1149 SpeedForward 21.917944 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X 0.006385002 Y 0 Z 0.019165885 Pitch 1.5695514489035771 Yaw -1.5104653947382445 Roll -1.5055223386492873 Position Submission#31+DataContainers+Vector3 X 492.36728 Y 50.008667 Z 624.0008 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 5 Submission#31+DataContainers+SampleData Time 0.5 RPM 2622.1104 SpeedForward 27.356396 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X 0.0062216795 Y 0 Z 0.019475697 Pitch 1.570308045540046 Yaw -1.5935587965990106 Roll -1.5863648243828028 Position Submission#31+DataContainers+Vector3 X 493.0555 Y 50.010757 Z 624.0008 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 6 Submission#31+DataContainers+SampleData Time 0.6 RPM 3202.106 SpeedForward 33.398876 SpeedSideward -0.015258789 AngularVelocity Submission#31+DataContainers+Vector3 X -0.0034594447 Y -0 Z 0.0020355275 Pitch 1.5707963267948966 Yaw -1.6478667982108606 Roll -1.6395197661140009 Position Submission#31+DataContainers+Vector3 X 493.90125 Y 50.01157 Z 624.0008 IsTurbo False FRGroundContactMaterial Asphalt FLGroundContactMaterial Asphalt RRGroundContactMaterial Asphalt RLGroundContactMaterial Asphalt 7 Submission#31+DataContainers+SampleData Time 0.7 RPM 3782.101 SpeedForward 38.83711 SpeedSideward -0.35095215 AngularVelocity Submission#31+DataContainers+Vector3 X 0.014734094 Y 0.16975488 Z 0.10025882 Pitch 1.569104870726493 Yaw -0.0866397650286374 Roll -0.08113132010948719 Position Submission#31+DataContainers+Vector3 X 494.90277 Y 50.008324

Error: System.ArgumentException: .NET number values such as positive and negative infinity cannot be written as valid JSON. To make it work when using 'JsonSerializer', consider specifying 'JsonNumberHandling.AllowNamedFloatingPointLiterals' (see https://docs.microsoft.com/dotnet/api/system.text.json.serialization.jsonnumberhandling).
   at System.Text.Json.ThrowHelper.ThrowArgumentException_ValueNotSupported()
   at System.Text.Json.Serialization.Metadata.JsonPropertyInfo`1.GetMemberAndWriteJson(Object obj, WriteStack& state, Utf8JsonWriter writer)
   at System.Text.Json.Serialization.Converters.ObjectDefaultConverter`1.OnTryWrite(Utf8JsonWriter writer, T value, JsonSerializerOptions options, WriteStack& state)
   at System.Text.Json.Serialization.JsonConverter`1.TryWrite(Utf8JsonWriter writer, T& value, JsonSerializerOptions options, WriteStack& state)
   at System.Text.Json.Serialization.Converters.ListOfTConverter`2.OnWriteResume(Utf8JsonWriter writer, TCollection value, JsonS